In [1]:
import pandas as pd
import geojson
import folium
import re
import numpy as np

### Cleaning functions

In [2]:

def cleancords(punto):
    '''Clean coord columns as it comes from scrapping'''
    try:
        punto = punto[2:-2]
        esplit = punto.split(',')
        esplite = (float(esplit[0]),float(esplit[1]))
        kords.append(esplite)
    except:
        kords.append(np.nan)

def cleandescr(crip):
    '''
    Transform the data in descr column to create 4 new columns with the contained integers.
    Requires the four list that are to become columns to be previously initialized
    '''
    hue = 0
    dorm = 0
    cam = 0
    toil = 0
    
    cripa = crip.split('·')

    for i in cripa:
        if 'pedes' in i:
            guest.append(float(re.findall('\d+', i)[0]))
            hue = 1
        elif 'orio' in i:
            try:
                bedroom.append(float(re.findall('\d+', i)[0]))
                dorm = 1
            except:
                pass
        elif 'cama' in i:
            try:
                beds.append(float(re.findall('\d+', i)[0]))
                cam = 1
            except:
                pass
        elif 'año' in i:
            try:
                bathroom.append(float(re.findall('\d+|\d+\,\d+', i)[0].replace(',','.')))
                toil = 1
            except:
                pass
        elif 'aseo' in i:
            try:
                bathroom.append(float(re.findall('\d+|\d+\,\d+', i)[0].replace(',','.')))
                toil = 1
            except:
                pass
        elif 'udio' in i:
            try:
                bedroom.append(1.5)
                dorm = 1
            except:
                pass
    if cam == 0:
        beds.append(np.nan)
    if dorm == 0:
        bedroom.append(np.nan)
    if toil == 0:
        bathroom.append(np.nan)
    if hue == 0:
        guest.append(np.nan)


def cleanprice(plo):
    '''Function that cleans price column'''
    plo.replace('.','')
    plo = re.findall('\d+', plo)[0]
    prise.append(float(plo))

In [3]:
# initialize list for cleandescr
guest = []
bedroom = []
beds = []
bathroom = []
# initialize list for cleancords
kords = [] 
# initialize list for cleanprice
prise = []

### Taking a look at the set

In [4]:
dftry = pd.read_csv('Scrapnight2_67_78.csv')
dftry.columns = ['id', 'title', 'url', 'descr', 'coord', 'price']
dftry.head()

,id,title,url,descr,coord,price
0,24938138,Balcón del mar,https://www.airbnb.es/rooms/24938138?adults=2&...,3 huéspedes · Estudio · 2 camas · 1 baño,"['28.5026,-16.4254']",42€
1,51940792,Surrounded by nature in front the Atlantic Oce...,https://www.airbnb.es/rooms/51940792?adults=2&...,2 huéspedes · 1 dormitorio · 2 camas · 1 baño,"['28.47853,-16.43751']",52€
2,10135375,"PARADISE BEACH & POOL, WIFI, VIEWS, TERRACE,GA...",https://www.airbnb.es/rooms/10135375?adults=2&...,3 huéspedes · 1 dormitorio · 2 camas · 1 baño,"['28.5033,-16.4229']",47€
3,3674557?,"THE VIEWS , The luxury oriental loft in the no...",https://www.airbnb.es/rooms/3674557?adults=2&c...,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,[],85€
4,17412527,"Apartamento frente al Océano. El Pris, Tacoronte.",https://www.airbnb.es/rooms/17412527?adults=2&...,6 huéspedes · 3 dormitorios · 5 camas · 1 baño,"['28.5091,-16.4214']",48€


### Cleaning the data

In [5]:
for e in list(dftry.coord):
    cleancords(e)
for e in list(dftry.descr):
    cleandescr(e)
for e in list(dftry.price):
    cleanprice(e)

dftry['prices'] = prise
dftry['guest'] = guest
dftry['bedrooms'] = bedroom
dftry['beds'] = beds
dftry['bathroom'] = bathroom
dftry['spot'] = kords

In [6]:
dftry.drop('price', axis=1, inplace=True)
dftry.drop('coord', axis=1, inplace=True)
dftry.drop('descr', axis=1, inplace=True)

In [7]:
dftry.head()

,id,title,url,prices,guest,bedrooms,beds,bathroom,spot
0,24938138,Balcón del mar,https://www.airbnb.es/rooms/24938138?adults=2&...,42.0,3.0,1.5,2.0,1.0,"(28.5026, -16.4254)"
1,51940792,Surrounded by nature in front the Atlantic Oce...,https://www.airbnb.es/rooms/51940792?adults=2&...,52.0,2.0,1.0,2.0,1.0,"(28.47853, -16.43751)"
2,10135375,"PARADISE BEACH & POOL, WIFI, VIEWS, TERRACE,GA...",https://www.airbnb.es/rooms/10135375?adults=2&...,47.0,3.0,1.0,2.0,1.0,"(28.5033, -16.4229)"
3,3674557?,"THE VIEWS , The luxury oriental loft in the no...",https://www.airbnb.es/rooms/3674557?adults=2&c...,85.0,2.0,1.0,1.0,1.0,NaN
4,17412527,"Apartamento frente al Océano. El Pris, Tacoronte.",https://www.airbnb.es/rooms/17412527?adults=2&...,48.0,6.0,3.0,5.0,1.0,"(28.5091, -16.4214)"


### Post cleaning exploration

In [8]:
dftry.dtypes

id           object
title        object
url          object
prices      float64
guest       float64
bedrooms    float64
beds        float64
bathroom    float64
spot         object
dtype: object

In [9]:
# ver si hay valores duplicados
boolean = dftry['id'].duplicated().any()
boolean

False

In [10]:
tnf = folium.Map(
    location=[28.273007, -16.640793])

for i in list(dftry['spot']):
    try:
        folium.Marker(i).add_to(tnf)
    except:
        pass
tnf